In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [2]:
# Define updated classes and functions
class Time_Series_Dataset(Dataset):
    def __init__(self, inputs, decoder_inputs, outputs):
        self.inputs = inputs
        self.decoder_inputs = decoder_inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        x = self.inputs[idx]
        decoder_input = self.decoder_inputs[idx]
        y = self.outputs[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(decoder_input, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        out, (hidden, cell) = self.lstm(x, (hidden, cell))
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_quantiles):
        super(Decoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size * num_quantiles)
        self.num_quantiles = num_quantiles

    def forward(self, x, hidden, cell):
        out, (hidden, cell) = self.lstm(x, (hidden, cell))
        out = self.fc(out)
        out = out.view(out.size(0), out.size(1), self.num_quantiles)
        return out, hidden, cell

class EncoderDecoderLSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers, num_quantiles):
        super(EncoderDecoderLSTM, self).__init__()
        self.encoder = Encoder(input_size, hidden_size)
        self.decoder = Decoder(output_size, hidden_size, output_size, num_quantiles)

    def forward(self, encoder_inputs, decoder_inputs):
        hidden, cell = self.encoder(encoder_inputs)
        decoder_inputs = decoder_inputs.unsqueeze(-1)  # Add feature dimension to decoder inputs
        outputs, hidden, cell = self.decoder(decoder_inputs, hidden, cell)
        return outputs

def quantile_loss(preds, targets, quantiles):
    losses = []
    for i, quantile in enumerate(quantiles):
        errors = targets[:, :, i] - preds[:, :, i]
        losses.append(torch.mean(torch.max((quantile - 1) * errors, quantile * errors)))
    return torch.mean(torch.stack(losses))

def split_data(features, target, input_steps, output_steps, train_ratio, seed):
    X, y, decoder_inputs = [], [], []
    total_size = input_steps + output_steps
    for i in range(len(features) - total_size + 1):
        X.append(features[i:i + input_steps])
        y.append(target[i + input_steps:i + total_size])
        decoder_inputs.append(target[i + input_steps - 1:i + input_steps + output_steps - 1])
    
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = train_test_split(
        X, y, decoder_inputs, train_size=train_ratio, random_state=seed
    )
    return X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test
    
pm = "\u00B1"

In [3]:
def lorenz(xyz, *, s=10, r=28, b=2.667):
    x, y, z = xyz
    x_dot = s*(y - x)
    y_dot = r*x - y - x*z
    z_dot = x*y - b*z
    return np.array([x_dot, y_dot, z_dot])

dt = 0.01
num_steps = 10000

xyzs = np.empty((num_steps + 1, 3))  # Need one more for the initial values
xyzs[0] = (0., 1., 1.05)  # Set initial values
# Step through "time", calculating the partial derivatives at the current point
# and using them to estimate the next point
for i in range(num_steps):
    xyzs[i + 1] = xyzs[i] + lorenz(xyzs[i]) * dt

# break it down to 3 univariate time series
x_dimension = xyzs[:, 0]
y_dimension = xyzs[:, 1]
z_dimension = xyzs[:, 2]

x_dimension = x_dimension.flatten()
x_dimension = pd.Series(x_dimension, name='Value')
y_dimension = y_dimension.flatten()
y_dimension = pd.Series(y_dimension, name='Value')
z_dimension = z_dimension.flatten()
z_dimension = pd.Series(z_dimension, name='Value')

x_dimension.index = range(len(x_dimension))
y_dimension.index = range(len(y_dimension))
z_dimension.index = range(len(z_dimension))
# print(x_dimension), print(y_dimension), print(z_dimension)

scaler = MinMaxScaler(feature_range=(0, 1))
x_reshaped = np.array(x_dimension).reshape(-1, 1)
y_reshaped = np.array(y_dimension).reshape(-1, 1)
z_reshaped = np.array(z_dimension).reshape(-1, 1)
x_scaled = scaler.fit_transform(x_reshaped).flatten()
y_scaled = scaler.fit_transform(y_reshaped).flatten()
z_scaled = scaler.fit_transform(z_reshaped).flatten()

In [4]:
input_steps = 5
output_steps = 10
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]
train_ratio = 0.8
seed = 5925
num_experiments = 30   # default: 30

rmse, mae, mape = [], [], []
rmse_005, rmse_025, rmse_050, rmse_075, rmse_095 = [], [], [], [], []
rmse_steps = [[] for _ in range(output_steps)]
mae_steps = [[] for _ in range(output_steps)]
mape_steps = [[] for _ in range(output_steps)]

train_rmse, train_mae, train_mape = [], [], []
train_rmse_steps = [[] for _ in range(output_steps)]
train_mae_steps = [[] for _ in range(output_steps)]
train_mape_steps = [[] for _ in range(output_steps)]

In [5]:
# X Dimension
features = x_scaled
target = x_scaled
for exp in range(num_experiments):
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = split_data(
        features, target, input_steps, output_steps, train_ratio, seed
    )
    
    train_dataset = Time_Series_Dataset(X_train, decoder_inputs_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, decoder_inputs_test, y_test)
    
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    # Define our parameters
    input_size = 1  
    output_size = 1  # Predicting one value per time step
    hidden_size = 100 # Default: 100
    num_layers = 2
    num_quantiles = len(quantiles)
    
    model = EncoderDecoderLSTM(input_size, output_size, hidden_size, num_layers, num_quantiles)
    
    # Loss and optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Training loop
    num_epochs = 100  # Default: 100
    for epoch in range(1, num_epochs + 1):
        model.train()
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
    
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
    
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
    
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, num_quantiles)
            targets = targets.repeat(1, 1, num_quantiles)  # Expand targets to match the number of quantiles
    
            # Compute loss
            loss = quantile_loss(outputs, targets, quantiles)
    
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluate the model on the test set
    model.eval()
    y_pred = []
    y_test = []
    
    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in test_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
    
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
    
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            outputs = outputs.view(batch_size, output_steps, num_quantiles)  # Reshape outputs
    
            y_pred.append(outputs.numpy())
            y_test.append(targets.repeat(1, 1, num_quantiles).numpy())  # Expand targets to match the number of quantiles
    
    # Convert lists to numpy arrays
    predicted_values = np.concatenate(y_pred, axis=0)
    actuals = np.concatenate(y_test, axis=0)[:, :, 0]

    pred_005 = predicted_values[:, :, 0]
    pred_025 = predicted_values[:, :, 1]
    pred_050 = predicted_values[:, :, 2]
    pred_075 = predicted_values[:, :, 3]
    pred_095 = predicted_values[:, :, 4]
    
    # Calculate RMSE here
    mse_005 = mean_squared_error(pred_005, actuals)
    mse_025 = mean_squared_error(pred_025, actuals)
    mse_050 = mean_squared_error(pred_050, actuals)
    mse_075 = mean_squared_error(pred_075, actuals)
    mse_095 = mean_squared_error(pred_095, actuals)

    rmse_005.append(np.sqrt(mse_005))
    rmse_025.append(np.sqrt(mse_025))
    rmse_050.append(np.sqrt(mse_050))
    rmse_075.append(np.sqrt(mse_075))
    rmse_095.append(np.sqrt(mse_095))

    pred_values = predicted_values.reshape(-1, 1)
    pred_values = scaler.inverse_transform(pred_values)
    pred_values = pred_values.reshape(predicted_values.shape)
    actual_values = scaler.inverse_transform(actuals)
    
    predicted_005 = pred_values[:, :, 0]
    predicted_025 = pred_values[:, :, 1]
    predicted_050 = pred_values[:, :, 2]
    predicted_075 = pred_values[:, :, 3]
    predicted_095 = pred_values[:, :, 4]

    mae.append(mean_absolute_error(actual_values, predicted_050))
    mape.append(mean_absolute_percentage_error(actual_values, predicted_050))

    for step in range(output_steps):
        mse_step = mean_squared_error(pred_050[:, step], actuals[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))
        mae_steps[step].append(mean_absolute_error(predicted_050[:, step], actual_values[:, step]))
        mape_steps[step].append(mean_absolute_percentage_error(predicted_050[:, step], actual_values[:, step]))
        
    # Evaluate the model on the train set
    y_train_pred = []
    y_train_actual = []

    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, num_quantiles)
            
            y_train_pred.append(outputs.numpy())
            y_train_actual.append(targets.repeat(1, 1, num_quantiles).numpy())  # Expand targets to match the number of quantiles

    # Convert lists to numpy arrays and remove last dimension
    y_train_pred = np.concatenate(y_train_pred, axis=0)
    y_train_actual = np.concatenate(y_train_actual, axis=0)

    # Calculate train metrics
    actuals_train = y_train_actual[:, :, 0]
    pred_train_050 = y_train_pred[:, :, 2]  # 0.50 quantile as the prediction

    train_mse = mean_squared_error(actuals_train, pred_train_050)
    train_rmse.append(np.sqrt(train_mse))

    for step in range(output_steps):
        train_mse_step = mean_squared_error(pred_train_050[:, step], actuals_train[:, step])
        train_rmse_steps[step].append(np.sqrt(train_mse_step))

    # Inverse Transform
    train_pred_values = y_train_pred.reshape(-1, 1)
    train_pred_values = scaler.inverse_transform(train_pred_values)
    train_pred_values = train_pred_values.reshape(y_train_pred.shape)
    train_actual_values = scaler.inverse_transform(actuals_train.reshape(-1, 1)).reshape(actuals_train.shape)
    
    train_predicted_050 = train_pred_values[:, :, 2]

    train_mae.append(mean_absolute_error(train_actual_values, train_predicted_050))
    train_mape.append(mean_absolute_percentage_error(train_actual_values, train_predicted_050))

    for step in range(output_steps):
        train_mae_steps[step].append(mean_absolute_error(train_actual_values[:, step], train_predicted_050[:, step]))
        train_mape_steps[step].append(mean_absolute_percentage_error(train_actual_values[:, step], train_predicted_050[:, step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1

Experiment 1/3 done
Experiment 2/3 done
Experiment 3/3 done


In [6]:
print(f"Lorenz X ED-LSTM Regression: After {num_experiments} experimental runs, here are the results:")

# Test dataset results
print(f"Across {output_steps} predictive time steps on the test dataset, " +
      f"Avg RMSE: {np.mean(rmse_050):.4f} {pm} {np.std(rmse_050):.4f}, " +
      f"Avg MAE: {np.mean(mae):.2f} {pm} {np.std(mae):.2f}, " +
      f"Avg MAPE: {np.mean(mape)*100:.3f}% {pm} {np.std(mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the test dataset, "
        f"Avg RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_steps[step]):.2f} {pm} {np.std(mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_steps[step]) * 100:.3f}% {pm} {np.std(mape_steps[step]) * 100:.3f}%"
    )

# Train dataset results
print(f"Across {output_steps} predictive time steps on the train dataset, " +
      f"Avg RMSE: {np.mean(train_rmse):.4f} {pm} {np.std(train_rmse):.4f}, " +
      f"Avg MAE: {np.mean(train_mae):.2f} {pm} {np.std(train_mae):.2f}, " +
      f"Avg MAPE: {np.mean(train_mape)*100:.3f}% {pm} {np.std(train_mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the train dataset, "
        f"Avg RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(train_mae_steps[step]):.2f} {pm} {np.std(train_mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(train_mape_steps[step]) * 100:.3f}% {pm} {np.std(train_mape_steps[step]) * 100:.3f}%")

Lorenz X ED-LSTM Regression: After 3 experimental runs, here are the results:
Across 10 predictive time steps on the test dataset, Avg RMSE: 0.5664 ± 0.1175, Avg MAE: 27.86 ± 6.72, Avg MAPE: 110.788% ± 35.589%
At time step 1 on the test dataset, Avg RMSE: 0.5797 ± 0.1346, Avg MAE: 28.77 ± 7.62, Avg MAPE: 1844.438% ± 2156.092%
At time step 2 on the test dataset, Avg RMSE: 0.5759 ± 0.1275, Avg MAE: 28.45 ± 7.22, Avg MAPE: 9866.666% ± 13444.475%
At time step 3 on the test dataset, Avg RMSE: 0.5703 ± 0.1203, Avg MAE: 28.09 ± 6.85, Avg MAPE: 27164.017% ± 37846.716%
At time step 4 on the test dataset, Avg RMSE: 0.5667 ± 0.1170, Avg MAE: 27.86 ± 6.70, Avg MAPE: 3575.130% ± 4437.171%
At time step 5 on the test dataset, Avg RMSE: 0.5643 ± 0.1153, Avg MAE: 27.72 ± 6.63, Avg MAPE: 2545.622% ± 2938.599%
At time step 6 on the test dataset, Avg RMSE: 0.5626 ± 0.1143, Avg MAE: 27.62 ± 6.59, Avg MAPE: 2139.841% ± 2328.864%
At time step 7 on the test dataset, Avg RMSE: 0.5615 ± 0.1137, Avg MAE: 27.56 ±

In [22]:
# Y-Dimension
input_steps = 5
output_steps = 10
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]
train_ratio = 0.8
seed = 5925
num_experiments = 30   # default: 30

rmse, mae, mape = [], [], []
rmse_005, rmse_025, rmse_050, rmse_075, rmse_095 = [], [], [], [], []
rmse_steps = [[] for _ in range(output_steps)]
mae_steps = [[] for _ in range(output_steps)]
mape_steps = [[] for _ in range(output_steps)]

train_rmse, train_mae, train_mape = [], [], []
train_rmse_steps = [[] for _ in range(output_steps)]
train_mae_steps = [[] for _ in range(output_steps)]
train_mape_steps = [[] for _ in range(output_steps)]

In [24]:
# Y Dimension
features = y_scaled
target = y_scaled
for exp in range(num_experiments):
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = split_data(
        features, target, input_steps, output_steps, train_ratio, seed
    )
    
    train_dataset = Time_Series_Dataset(X_train, decoder_inputs_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, decoder_inputs_test, y_test)
    
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    # Define our parameters
    input_size = 1  
    output_size = 1  # Predicting one value per time step
    hidden_size = 100 # Default: 100
    num_layers = 2
    num_quantiles = len(quantiles)
    
    model = EncoderDecoderLSTM(input_size, output_size, hidden_size, num_layers, num_quantiles)
    
    # Loss and optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Training loop
    num_epochs = 100  # Default: 100
    for epoch in range(1, num_epochs + 1):
        model.train()
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
    
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
    
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
    
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, num_quantiles)
            targets = targets.repeat(1, 1, num_quantiles)  # Expand targets to match the number of quantiles
    
            # Compute loss
            loss = quantile_loss(outputs, targets, quantiles)
    
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluate the model on the test set
    model.eval()
    y_pred = []
    y_test = []
    
    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in test_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
    
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
    
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            outputs = outputs.view(batch_size, output_steps, num_quantiles)  # Reshape outputs
    
            y_pred.append(outputs.numpy())
            y_test.append(targets.repeat(1, 1, num_quantiles).numpy())  # Expand targets to match the number of quantiles
    
    # Convert lists to numpy arrays
    predicted_values = np.concatenate(y_pred, axis=0)
    actuals = np.concatenate(y_test, axis=0)[:, :, 0]

    pred_005 = predicted_values[:, :, 0]
    pred_025 = predicted_values[:, :, 1]
    pred_050 = predicted_values[:, :, 2]
    pred_075 = predicted_values[:, :, 3]
    pred_095 = predicted_values[:, :, 4]
    
    # Calculate RMSE here
    mse_005 = mean_squared_error(pred_005, actuals)
    mse_025 = mean_squared_error(pred_025, actuals)
    mse_050 = mean_squared_error(pred_050, actuals)
    mse_075 = mean_squared_error(pred_075, actuals)
    mse_095 = mean_squared_error(pred_095, actuals)

    rmse_005.append(np.sqrt(mse_005))
    rmse_025.append(np.sqrt(mse_025))
    rmse_050.append(np.sqrt(mse_050))
    rmse_075.append(np.sqrt(mse_075))
    rmse_095.append(np.sqrt(mse_095))

    pred_values = predicted_values.reshape(-1, 1)
    pred_values = scaler.inverse_transform(pred_values)
    pred_values = pred_values.reshape(predicted_values.shape)
    actual_values = scaler.inverse_transform(actuals)
    
    predicted_005 = pred_values[:, :, 0]
    predicted_025 = pred_values[:, :, 1]
    predicted_050 = pred_values[:, :, 2]
    predicted_075 = pred_values[:, :, 3]
    predicted_095 = pred_values[:, :, 4]

    mae.append(mean_absolute_error(actual_values, predicted_050))
    mape.append(mean_absolute_percentage_error(actual_values, predicted_050))

    for step in range(output_steps):
        mse_step = mean_squared_error(pred_050[:, step], actuals[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))
        mae_steps[step].append(mean_absolute_error(predicted_050[:, step], actual_values[:, step]))
        mape_steps[step].append(mean_absolute_percentage_error(predicted_050[:, step], actual_values[:, step]))
        
    # Evaluate the model on the train set
    y_train_pred = []
    y_train_actual = []

    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, num_quantiles)
            
            y_train_pred.append(outputs.numpy())
            y_train_actual.append(targets.repeat(1, 1, num_quantiles).numpy())  # Expand targets to match the number of quantiles

    # Convert lists to numpy arrays and remove last dimension
    y_train_pred = np.concatenate(y_train_pred, axis=0)
    y_train_actual = np.concatenate(y_train_actual, axis=0)

    # Calculate train metrics
    actuals_train = y_train_actual[:, :, 0]
    pred_train_050 = y_train_pred[:, :, 2]  # 0.50 quantile as the prediction

    train_mse = mean_squared_error(actuals_train, pred_train_050)
    train_rmse.append(np.sqrt(train_mse))

    for step in range(output_steps):
        train_mse_step = mean_squared_error(pred_train_050[:, step], actuals_train[:, step])
        train_rmse_steps[step].append(np.sqrt(train_mse_step))

    # Inverse Transform
    train_pred_values = y_train_pred.reshape(-1, 1)
    train_pred_values = scaler.inverse_transform(train_pred_values)
    train_pred_values = train_pred_values.reshape(y_train_pred.shape)
    train_actual_values = scaler.inverse_transform(actuals_train.reshape(-1, 1)).reshape(actuals_train.shape)
    
    train_predicted_050 = train_pred_values[:, :, 2]

    train_mae.append(mean_absolute_error(train_actual_values, train_predicted_050))
    train_mape.append(mean_absolute_percentage_error(train_actual_values, train_predicted_050))

    for step in range(output_steps):
        train_mae_steps[step].append(mean_absolute_error(train_actual_values[:, step], train_predicted_050[:, step]))
        train_mape_steps[step].append(mean_absolute_percentage_error(train_actual_values[:, step], train_predicted_050[:, step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1

Experiment 1/3 done
Experiment 2/3 done
Experiment 3/3 done


In [25]:
print(f"Lorenz Y ED-LSTM Regression: After {num_experiments} experimental runs, here are the results:")

# Test dataset results
print(f"Across {output_steps} predictive time steps on the test dataset, " +
      f"Avg RMSE: {np.mean(rmse_050):.4f} {pm} {np.std(rmse_050):.4f}, " +
      f"Avg MAE: {np.mean(mae):.2f} {pm} {np.std(mae):.2f}, " +
      f"Avg MAPE: {np.mean(mape)*100:.3f}% {pm} {np.std(mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the test dataset, "
        f"Avg RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_steps[step]):.2f} {pm} {np.std(mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_steps[step]) * 100:.3f}% {pm} {np.std(mape_steps[step]) * 100:.3f}%"
    )

# Train dataset results
print(f"Across {output_steps} predictive time steps on the train dataset, " +
      f"Avg RMSE: {np.mean(train_rmse):.4f} {pm} {np.std(train_rmse):.4f}, " +
      f"Avg MAE: {np.mean(train_mae):.2f} {pm} {np.std(train_mae):.2f}, " +
      f"Avg MAPE: {np.mean(train_mape)*100:.3f}% {pm} {np.std(train_mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the train dataset, "
        f"Avg RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(train_mae_steps[step]):.2f} {pm} {np.std(train_mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(train_mape_steps[step]) * 100:.3f}% {pm} {np.std(train_mape_steps[step]) * 100:.3f}%")

Lorenz Y ED-LSTM Regression: After 3 experimental runs, here are the results:
Across 10 predictive time steps on the test dataset, Avg RMSE: 0.6611 ± 0.0561, Avg MAE: 33.80 ± 2.95, Avg MAPE: 140.725% ± 12.715%
At time step 1 on the test dataset, Avg RMSE: 0.6325 ± 0.0414, Avg MAE: 32.18 ± 2.26, Avg MAPE: 522.395% ± 124.503%
At time step 2 on the test dataset, Avg RMSE: 0.6565 ± 0.0499, Avg MAE: 33.52 ± 2.65, Avg MAPE: 476.500% ± 170.071%
At time step 3 on the test dataset, Avg RMSE: 0.6630 ± 0.0574, Avg MAE: 33.90 ± 3.03, Avg MAPE: 478.399% ± 194.126%
At time step 4 on the test dataset, Avg RMSE: 0.6648 ± 0.0604, Avg MAE: 34.01 ± 3.19, Avg MAPE: 481.975% ± 204.412%
At time step 5 on the test dataset, Avg RMSE: 0.6653 ± 0.0615, Avg MAE: 34.05 ± 3.25, Avg MAPE: 484.092% ± 208.751%
At time step 6 on the test dataset, Avg RMSE: 0.6654 ± 0.0619, Avg MAE: 34.06 ± 3.27, Avg MAPE: 485.079% ± 210.493%
At time step 7 on the test dataset, Avg RMSE: 0.6654 ± 0.0620, Avg MAE: 34.07 ± 3.28, Avg MAPE

In [28]:
# Z-Dimension
input_steps = 5
output_steps = 10
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]
train_ratio = 0.8
seed = 5925
num_experiments = 30   # default: 30

rmse, mae, mape = [], [], []
rmse_005, rmse_025, rmse_050, rmse_075, rmse_095 = [], [], [], [], []
rmse_steps = [[] for _ in range(output_steps)]
mae_steps = [[] for _ in range(output_steps)]
mape_steps = [[] for _ in range(output_steps)]

train_rmse, train_mae, train_mape = [], [], []
train_rmse_steps = [[] for _ in range(output_steps)]
train_mae_steps = [[] for _ in range(output_steps)]
train_mape_steps = [[] for _ in range(output_steps)]

In [30]:
# Z Dimension
features = z_scaled
target = z_scaled
for exp in range(num_experiments):
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = split_data(
        features, target, input_steps, output_steps, train_ratio, seed
    )
    
    train_dataset = Time_Series_Dataset(X_train, decoder_inputs_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, decoder_inputs_test, y_test)
    
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    # Define our parameters
    input_size = 1  
    output_size = 1  # Predicting one value per time step
    hidden_size = 100 # Default: 100
    num_layers = 2
    num_quantiles = len(quantiles)
    
    model = EncoderDecoderLSTM(input_size, output_size, hidden_size, num_layers, num_quantiles)
    
    # Loss and optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Training loop
    num_epochs = 100  # Default: 100
    for epoch in range(1, num_epochs + 1):
        model.train()
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
    
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
    
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
    
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, num_quantiles)
            targets = targets.repeat(1, 1, num_quantiles)  # Expand targets to match the number of quantiles
    
            # Compute loss
            loss = quantile_loss(outputs, targets, quantiles)
    
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluate the model on the test set
    model.eval()
    y_pred = []
    y_test = []
    
    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in test_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
    
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
    
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            outputs = outputs.view(batch_size, output_steps, num_quantiles)  # Reshape outputs
    
            y_pred.append(outputs.numpy())
            y_test.append(targets.repeat(1, 1, num_quantiles).numpy())  # Expand targets to match the number of quantiles
    
    # Convert lists to numpy arrays
    predicted_values = np.concatenate(y_pred, axis=0)
    actuals = np.concatenate(y_test, axis=0)[:, :, 0]

    pred_005 = predicted_values[:, :, 0]
    pred_025 = predicted_values[:, :, 1]
    pred_050 = predicted_values[:, :, 2]
    pred_075 = predicted_values[:, :, 3]
    pred_095 = predicted_values[:, :, 4]
    
    # Calculate RMSE here
    mse_005 = mean_squared_error(pred_005, actuals)
    mse_025 = mean_squared_error(pred_025, actuals)
    mse_050 = mean_squared_error(pred_050, actuals)
    mse_075 = mean_squared_error(pred_075, actuals)
    mse_095 = mean_squared_error(pred_095, actuals)

    rmse_005.append(np.sqrt(mse_005))
    rmse_025.append(np.sqrt(mse_025))
    rmse_050.append(np.sqrt(mse_050))
    rmse_075.append(np.sqrt(mse_075))
    rmse_095.append(np.sqrt(mse_095))

    pred_values = predicted_values.reshape(-1, 1)
    pred_values = scaler.inverse_transform(pred_values)
    pred_values = pred_values.reshape(predicted_values.shape)
    actual_values = scaler.inverse_transform(actuals)
    
    predicted_005 = pred_values[:, :, 0]
    predicted_025 = pred_values[:, :, 1]
    predicted_050 = pred_values[:, :, 2]
    predicted_075 = pred_values[:, :, 3]
    predicted_095 = pred_values[:, :, 4]

    mae.append(mean_absolute_error(actual_values, predicted_050))
    mape.append(mean_absolute_percentage_error(actual_values, predicted_050))

    for step in range(output_steps):
        mse_step = mean_squared_error(pred_050[:, step], actuals[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))
        mae_steps[step].append(mean_absolute_error(predicted_050[:, step], actual_values[:, step]))
        mape_steps[step].append(mean_absolute_percentage_error(predicted_050[:, step], actual_values[:, step]))
        
    # Evaluate the model on the train set
    y_train_pred = []
    y_train_actual = []

    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, num_quantiles)
            
            y_train_pred.append(outputs.numpy())
            y_train_actual.append(targets.repeat(1, 1, num_quantiles).numpy())  # Expand targets to match the number of quantiles

    # Convert lists to numpy arrays and remove last dimension
    y_train_pred = np.concatenate(y_train_pred, axis=0)
    y_train_actual = np.concatenate(y_train_actual, axis=0)

    # Calculate train metrics
    actuals_train = y_train_actual[:, :, 0]
    pred_train_050 = y_train_pred[:, :, 2]  # 0.50 quantile as the prediction

    train_mse = mean_squared_error(actuals_train, pred_train_050)
    train_rmse.append(np.sqrt(train_mse))

    for step in range(output_steps):
        train_mse_step = mean_squared_error(pred_train_050[:, step], actuals_train[:, step])
        train_rmse_steps[step].append(np.sqrt(train_mse_step))

    # Inverse Transform
    train_pred_values = y_train_pred.reshape(-1, 1)
    train_pred_values = scaler.inverse_transform(train_pred_values)
    train_pred_values = train_pred_values.reshape(y_train_pred.shape)
    train_actual_values = scaler.inverse_transform(actuals_train.reshape(-1, 1)).reshape(actuals_train.shape)
    
    train_predicted_050 = train_pred_values[:, :, 2]

    train_mae.append(mean_absolute_error(train_actual_values, train_predicted_050))
    train_mape.append(mean_absolute_percentage_error(train_actual_values, train_predicted_050))

    for step in range(output_steps):
        train_mae_steps[step].append(mean_absolute_error(train_actual_values[:, step], train_predicted_050[:, step]))
        train_mape_steps[step].append(mean_absolute_percentage_error(train_actual_values[:, step], train_predicted_050[:, step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1

Experiment 1/3 done
Experiment 2/3 done
Experiment 3/3 done


In [31]:
print(f"Lorenz Z ED-LSTM Regression: After {num_experiments} experimental runs, here are the results:")

# Test dataset results
print(f"Across {output_steps} predictive time steps on the test dataset, " +
      f"Avg RMSE: {np.mean(rmse_050):.4f} {pm} {np.std(rmse_050):.4f}, " +
      f"Avg MAE: {np.mean(mae):.2f} {pm} {np.std(mae):.2f}, " +
      f"Avg MAPE: {np.mean(mape)*100:.3f}% {pm} {np.std(mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the test dataset, "
        f"Avg RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_steps[step]):.2f} {pm} {np.std(mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_steps[step]) * 100:.3f}% {pm} {np.std(mape_steps[step]) * 100:.3f}%"
    )

# Train dataset results
print(f"Across {output_steps} predictive time steps on the train dataset, " +
      f"Avg RMSE: {np.mean(train_rmse):.4f} {pm} {np.std(train_rmse):.4f}, " +
      f"Avg MAE: {np.mean(train_mae):.2f} {pm} {np.std(train_mae):.2f}, " +
      f"Avg MAPE: {np.mean(train_mape)*100:.3f}% {pm} {np.std(train_mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the train dataset, "
        f"Avg RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(train_mae_steps[step]):.2f} {pm} {np.std(train_mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(train_mape_steps[step]) * 100:.3f}% {pm} {np.std(train_mape_steps[step]) * 100:.3f}%")

Lorenz Z ED-LSTM Regression: After 3 experimental runs, here are the results:
Across 10 predictive time steps on the test dataset, Avg RMSE: 0.6978 ± 0.3945, Avg MAE: 36.01 ± 21.35, Avg MAPE: 157.141% ± 90.005%
At time step 1 on the test dataset, Avg RMSE: 0.7421 ± 0.4232, Avg MAE: 38.42 ± 22.84, Avg MAPE: 132.456% ± 75.443%
At time step 2 on the test dataset, Avg RMSE: 0.7172 ± 0.4084, Avg MAE: 37.09 ± 22.08, Avg MAPE: 136.605% ± 80.031%
At time step 3 on the test dataset, Avg RMSE: 0.7024 ± 0.3989, Avg MAE: 36.28 ± 21.59, Avg MAPE: 139.907% ± 83.153%
At time step 4 on the test dataset, Avg RMSE: 0.6942 ± 0.3932, Avg MAE: 35.84 ± 21.30, Avg MAPE: 142.049% ± 85.024%
At time step 5 on the test dataset, Avg RMSE: 0.6897 ± 0.3899, Avg MAE: 35.59 ± 21.12, Avg MAPE: 143.319% ± 86.073%
At time step 6 on the test dataset, Avg RMSE: 0.6873 ± 0.3880, Avg MAE: 35.46 ± 21.02, Avg MAPE: 144.043% ± 86.629%
At time step 7 on the test dataset, Avg RMSE: 0.6861 ± 0.3868, Avg MAE: 35.39 ± 20.96, Avg MA